In [58]:
import pandas as pd
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [59]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
#load all xml files in a list
xmlfiles = glob("./data_images/*.xml")
#datacleaning
replace_text = lambda x:x.replace("\\","/")
xmlfiles = list(map(replace_text,xmlfiles))

xmlfiles

['./data_images/009032.xml',
 './data_images/007544.xml',
 './data_images/009523.xml',
 './data_images/007537.xml',
 './data_images/008976.xml',
 './data_images/001544.xml',
 './data_images/007873.xml',
 './data_images/004185.xml',
 './data_images/001604.xml',
 './data_images/008773.xml',
 './data_images/007655.xml',
 './data_images/007649.xml',
 './data_images/005208.xml',
 './data_images/007336.xml',
 './data_images/004281.xml',
 './data_images/004033.xml',
 './data_images/000320.xml',
 './data_images/006135.xml',
 './data_images/005181.xml',
 './data_images/003011.xml',
 './data_images/007697.xml',
 './data_images/002496.xml',
 './data_images/004229.xml',
 './data_images/004671.xml',
 './data_images/009029.xml',
 './data_images/005591.xml',
 './data_images/000220.xml',
 './data_images/009454.xml',
 './data_images/002153.xml',
 './data_images/002501.xml',
 './data_images/006524.xml',
 './data_images/004356.xml',
 './data_images/008241.xml',
 './data_images/008878.xml',
 './data_image

In [61]:
# step-2: read xml files
# from each xml file we need to extract
# filename, size(width, height), object(name, xmin, xmax, ymin, ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # extract filename
    image_name = root.find('filename').text
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name,xmin,xmax,ymin,ymax])
        
    return parser

In [62]:
parser_all = list(map(extract_text,xmlfiles))

In [63]:
data = reduce(lambda x, y : x+y, parser_all)

In [64]:
df = pd.DataFrame(data,columns = ["filename","width","height","name","xmin","xmax","ymin","ymax"])

In [65]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,009032.jpg,500,375,pottedplant,309,341,209,257
1,009032.jpg,500,375,tvmonitor,27,207,69,262
2,009032.jpg,500,375,tvmonitor,205,350,75,256
3,007544.jpg,500,375,tvmonitor,294,341,109,142
4,007544.jpg,500,375,chair,340,482,163,311


In [66]:
df.shape

(15663, 8)

In [67]:
df["name"].value_counts()

name
person         5447
car            1650
chair          1427
bottle          634
pottedplant     625
bird            599
dog             538
sofa            425
bicycle         418
horse           406
boat            398
motorbike       390
cat             389
tvmonitor       367
cow             356
sheep           353
aeroplane       331
train           328
diningtable     310
bus             272
Name: count, dtype: int64

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15663 entries, 0 to 15662
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  15663 non-null  object
 1   width     15663 non-null  object
 2   height    15663 non-null  object
 3   name      15663 non-null  object
 4   xmin      15663 non-null  object
 5   xmax      15663 non-null  object
 6   ymin      15663 non-null  object
 7   ymax      15663 non-null  object
dtypes: object(8)
memory usage: 979.1+ KB


In [69]:
# type conversion
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15663 entries, 0 to 15662
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  15663 non-null  object
 1   width     15663 non-null  int64 
 2   height    15663 non-null  int64 
 3   name      15663 non-null  object
 4   xmin      15663 non-null  int64 
 5   xmax      15663 non-null  int64 
 6   ymin      15663 non-null  int64 
 7   ymax      15663 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 979.1+ KB


In [70]:
# center x, center y
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
# w 
df['w'] = (df['xmax']-df['xmin'])/df['width']
# h 
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [71]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,009032.jpg,500,375,pottedplant,309,341,209,257,0.650,0.621333,0.064,0.128000
1,009032.jpg,500,375,tvmonitor,27,207,69,262,0.234,0.441333,0.360,0.514667
2,009032.jpg,500,375,tvmonitor,205,350,75,256,0.555,0.441333,0.290,0.482667
3,007544.jpg,500,375,tvmonitor,294,341,109,142,0.635,0.334667,0.094,0.088000
4,007544.jpg,500,375,chair,340,482,163,311,0.822,0.632000,0.284,0.394667


# split data into train and test

In [72]:
images = df['filename'].unique()

In [73]:
len(images)

5012

In [74]:
# 80% train and 20% test
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename']) # shuffle and pick 80% of images

In [75]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [76]:
len(img_train),len(img_test)

(4010, 1002)

In [77]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [78]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,009032.jpg,500,375,pottedplant,309,341,209,257,0.650,0.621333,0.064,0.128000
1,009032.jpg,500,375,tvmonitor,27,207,69,262,0.234,0.441333,0.360,0.514667
2,009032.jpg,500,375,tvmonitor,205,350,75,256,0.555,0.441333,0.290,0.482667
3,007544.jpg,500,375,tvmonitor,294,341,109,142,0.635,0.334667,0.094,0.088000
4,007544.jpg,500,375,chair,340,482,163,311,0.822,0.632000,0.284,0.394667


In [79]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
16,001544.jpg,500,375,person,1,440,1,375,0.441000,0.501333,0.87800,0.997333
28,005208.jpg,500,333,person,111,231,156,333,0.342000,0.734234,0.24000,0.531532
60,002496.jpg,500,404,train,61,500,70,348,0.561000,0.517327,0.87800,0.688119
63,004671.jpg,317,500,bottle,49,283,1,497,0.523659,0.498000,0.73817,0.992000
77,000220.jpg,500,375,car,40,464,127,292,0.504000,0.558667,0.84800,0.440000


In [80]:
# label encoding
def label_encoding(x):
    labels = {'person':0, 'car':1, 'chair':2, 'bottle':3, 'pottedplant':4, 'bird':5, 'dog':6,
       'sofa':7, 'bicycle':8, 'horse':9, 'boat':10, 'motorbike':11, 'cat':12, 'tvmonitor':13,
       'cow':14, 'sheep':15, 'aeroplane':16, 'train':17, 'diningtable':18, 'bus':19}
    return labels[x]

In [81]:
# Crear una copia de los DataFrames de entrenamiento y prueba
# Aplicar la codificación de etiquetas en las copias
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

In [82]:
train_df.head(10)

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,009032.jpg,500,375,pottedplant,309,341,209,257,0.650,0.621333,0.064,0.128000,4
1,009032.jpg,500,375,tvmonitor,27,207,69,262,0.234,0.441333,0.360,0.514667,13
2,009032.jpg,500,375,tvmonitor,205,350,75,256,0.555,0.441333,0.290,0.482667,13
3,007544.jpg,500,375,tvmonitor,294,341,109,142,0.635,0.334667,0.094,0.088000,13
4,007544.jpg,500,375,chair,340,482,163,311,0.822,0.632000,0.284,0.394667,2
5,007544.jpg,500,375,chair,190,274,188,344,0.464,0.709333,0.168,0.416000,2
6,007544.jpg,500,375,chair,124,192,190,347,0.316,0.716000,0.136,0.418667,2
7,007544.jpg,500,375,chair,54,132,206,372,0.186,0.770667,0.156,0.442667,2
8,007544.jpg,500,375,chair,57,118,177,222,0.175,0.532000,0.122,0.120000,2
9,007544.jpg,500,375,chair,185,221,171,217,0.406,0.517333,0.072,0.122667,2


In [83]:
import os
from shutil import move

train_folder = 'data_images/train'
test_folder = 'data_images/test'


os.mkdir(train_folder)
os.mkdir(test_folder)

FileExistsError: [Errno 17] File exists: 'data_images/train'

In [ ]:
cols = ['filename','id','center_x','center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')


In [94]:
#groupby_obj_train.get_group('000009.jpg').set_index('filename').to_csv('sample.txt',index=False,header=False)
# save each image in train/test folder and respectie labels in .txt

def save_data(filename, folder_path, group_obj):
    # move image
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path,filename)
    move(src,dst) # move image to the destination folder
    
    # save the labels
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)
    

In [95]:
filename_series = pd.Series(groupby_obj_train.groups.keys())
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

0       None
1       None
2       None
3       None
4       None
        ... 
4005    None
4006    None
4007    None
4008    None
4009    None
Length: 4010, dtype: object

In [96]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

0       None
1       None
2       None
3       None
4       None
        ... 
997     None
998     None
999     None
1000    None
1001    None
Length: 1002, dtype: object